# Capstone Project - The Battle of Neighborhoods

# Brooklyn Food Truck Location Recommendations

# I. PURPOSE

This document provides the details of my peer reviewed assignment for the IBM Data Science Professional Certificate Capstone project.

# II. INTRODUCTION

New York City is the largest city in U.S.A by population, which also encompasses five divisions called boroughs.  Of the five boroughs we will focus on Brooklyn, which is known for its night life and great food.

# III. Objective

In this project, we will studies the areas of Brooklyn in order to determine which neighborhoods have the highest concentration of bars in order to solve our business problem.  

## A description of the problem and discussion of the background. (15 marks)

Our business problem is that we are working with a Food Truck business owner who is trying to determine where he should take his food truck for the greatest chance of success.<br/>
The aim of this project is to segment area's of Brooklyn's neighborhoods based on the concentration of bars using the Foursquare data.  <br/>We therorize that the best places to take a food truck at night is near areas that have high concentrations of bars.<br/>  This theory is based on two main principals.<br/>
1. Not all bars have a kitchen and sell food, so targeting these areas fills a need for our customers.<br/>
2. People going home from the bars are more likely to buy food to enjoy late at night when many other food options are closed.<br/>

Thus the area's with the highest concentration of bars will have the highest concentrations of potential customers.<br/>




## A description of the data and how it will be used to solve the problem. (15 marks)

We will be using the Foursquare Places API for our data source. https://developer.foursquare.com/docs/api<br/>
From this API we will use the Venue Categories and Venue Location data to map our data on a Folium map.<br/>
From here we will use K-Means clustering to find the ideal locations to send the Food Trucks to.

# III. CODE

### Import necessary Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Define Foursquare Credentials and Version

##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [2]:
# @hidden
CLIENT_ID = 'CVFZIHHKFIYX5VCCCJJLKGBEKGTCXA2VLQOCMFTXJNFRMIS2' # your Foursquare ID
CLIENT_SECRET = 'IHUGWTRHYEUYIZOP1UAMJKI1FAWCPHI5H22SAHAR1WGF1TXZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CVFZIHHKFIYX5VCCCJJLKGBEKGTCXA2VLQOCMFTXJNFRMIS2
CLIENT_SECRET:IHUGWTRHYEUYIZOP1UAMJKI1FAWCPHI5H22SAHAR1WGF1TXZ


#### Let's again assume that you are staying in Brooklyn. So let's start by converting a Brooklyn address to its latitude and longitude coordinates.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [3]:
address = '156 North 4th St, Brooklyn, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7157208888889 -73.9589547037037


## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

In [4]:
search_query = 'Bar'
radius = 500
print(search_query + ' .... OK!')

Bar .... OK!


#### Define the corresponding URL

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=CVFZIHHKFIYX5VCCCJJLKGBEKGTCXA2VLQOCMFTXJNFRMIS2&client_secret=IHUGWTRHYEUYIZOP1UAMJKI1FAWCPHI5H22SAHAR1WGF1TXZ&ll=40.7157208888889,-73.9589547037037&v=20180604&query=Bar&radius=500&limit=30'

#### Send the GET Request and examine the results

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5da078c6c58ed7002cd0b6c9'},
 'response': {'venues': [{'id': '4581734ff964a520653f1fe3',
    'name': 'Surf Bar',
    'location': {'address': '139 N 6th St',
     'crossStreet': 'btwn Bedford & Berry St.',
     'lat': 40.71763628703034,
     'lng': -73.95871390484886,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71763628703034,
       'lng': -73.95871390484886}],
     'distance': 214,
     'postalCode': '11249',
     'cc': 'US',
     'city': 'Brooklyn',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['139 N 6th St (btwn Bedford & Berry St.)',
      'Brooklyn, NY 11249',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ce941735',
      'name': 'Seafood Restaurant',
      'pluralName': 'Seafood Restaurants',
      'shortName': 'Seafood',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",300434,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/surf-bar-139-n-6...,False,4581734ff964a520653f1fe3,139 N 6th St,...,"[139 N 6th St (btwn Bedford & Berry St.), Broo...","[{'label': 'display', 'lat': 40.71763628703034...",40.717636,-73.958714,NaN,11249,NY,Surf Bar,v-1570797766,78812387
1,"[{'id': '4bf58dd8d48988d152941735', 'name': 'A...",NaN,NaN,NaN,NaN,NaN,NaN,False,49bb36fcf964a520dc531fe3,291 Grand St,...,"[291 Grand St (btwn Roebling & Havemeyer St), ...","[{'label': 'display', 'lat': 40.71307533802476...",40.713075,-73.957462,NaN,11211,NY,Caracas Arepa Bar,v-1570797766,NaN
2,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",316509,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/teddys-bar-and-g...,False,4262f880f964a52025211fe3,96 Berry St,...,"[96 Berry St (at N. 8th), Brooklyn, NY 11249, ...","[{'label': 'display', 'lat': 40.71920479788144...",40.719205,-73.958431,NaN,11249,NY,Teddy's Bar & Grill,v-1570797766,78597371
3,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4af33812f964a520d2eb21e3,318 Grand St,...,[318 Grand St (btwn Havemeyer St. & Marcy Ave....,"[{'label': 'display', 'lat': 40.71266183593462...",40.712662,-73.956688,NaN,11211,NY,Full Circle Bar,v-1570797766,NaN
4,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",290980,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/doc-wine-bar-83-...,False,4f825769e4b024a26f122b5d,83 N 7th St,...,"[83 N 7th St (at Wythe Ave), Brooklyn, NY 1124...","[{'label': 'display', 'lat': 40.71958137689327...",40.719581,-73.960445,NaN,11249,NY,D.O.C. Wine Bar,v-1570797766,47216953


#### Define information of interest and filter dataframe

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Surf Bar,Seafood Restaurant,139 N 6th St,US,Brooklyn,United States,btwn Bedford & Berry St.,214,"[139 N 6th St (btwn Bedford & Berry St.), Broo...","[{'label': 'display', 'lat': 40.71763628703034...",40.717636,-73.958714,NaN,11249,NY,4581734ff964a520653f1fe3
1,Caracas Arepa Bar,Arepa Restaurant,291 Grand St,US,Brooklyn,United States,btwn Roebling & Havemeyer St,320,"[291 Grand St (btwn Roebling & Havemeyer St), ...","[{'label': 'display', 'lat': 40.71307533802476...",40.713075,-73.957462,NaN,11211,NY,49bb36fcf964a520dc531fe3
2,Teddy's Bar & Grill,Pub,96 Berry St,US,Brooklyn,United States,at N. 8th,390,"[96 Berry St (at N. 8th), Brooklyn, NY 11249, ...","[{'label': 'display', 'lat': 40.71920479788144...",40.719205,-73.958431,NaN,11249,NY,4262f880f964a52025211fe3
3,Full Circle Bar,Bar,318 Grand St,US,Brooklyn,United States,btwn Havemeyer St. & Marcy Ave.,390,[318 Grand St (btwn Havemeyer St. & Marcy Ave....,"[{'label': 'display', 'lat': 40.71266183593462...",40.712662,-73.956688,NaN,11211,NY,4af33812f964a520d2eb21e3
4,D.O.C. Wine Bar,Italian Restaurant,83 N 7th St,US,Brooklyn,United States,at Wythe Ave,447,"[83 N 7th St (at Wythe Ave), Brooklyn, NY 1124...","[{'label': 'display', 'lat': 40.71958137689327...",40.719581,-73.960445,NaN,11249,NY,4f825769e4b024a26f122b5d
5,Dim Sum Bar,Dim Sum Restaurant,167 Grand St,US,Brooklyn,United States,at Bedford Ave,241,"[167 Grand St (at Bedford Ave), Brooklyn, NY 1...","[{'label': 'display', 'lat': 40.71455662668249...",40.714557,-73.961375,NaN,11249,NY,55e6598a498e18adc16896db
6,North 4 Bar,Bar,160 N 4th St,US,Brooklyn,United States,at Bedford,8,"[160 N 4th St (at Bedford), Brooklyn, NY 11211...","[{'label': 'display', 'lat': 40.715796, 'lng':...",40.715796,-73.958950,NaN,11211,NY,4a7f9a5cf964a5205ff41fe3
7,Dardy Bar,Dive Bar,245 S. 1st Street,US,Brooklyn,United States,Roebling,354,"[245 S. 1st Street (Roebling), Brooklyn, NY 11...","[{'label': 'display', 'lat': 40.712627, 'lng':...",40.712627,-73.957970,NaN,11211,NY,54544a09498ea8054f640543
8,Momofuku Milk Bar,Dessert Shop,382 Metropolitan Ave,US,Brooklyn,United States,btwn Havemeyer & Marcy,358,[382 Metropolitan Ave (btwn Havemeyer & Marcy)...,"[{'label': 'display', 'lat': 40.71389, 'lng': ...",40.713890,-73.955455,NaN,11211,NY,4cfbed25d8468cfacd6df76b
9,Woodhul Wine Bar,Wine Bar,644 Driggs Ave,US,Brooklyn,United States,Metropolitan Ave,96,"[644 Driggs Ave (Metropolitan Ave), Brooklyn, ...","[{'label': 'display', 'lat': 40.71486379874682...",40.714864,-73.958760,Williamsburg,11211,NY,545c3ea6498ee02970f218d4


#### Let's visualize the Bars that are nearby

In [9]:
dataframe_filtered.name

0                                              Surf Bar
1                                     Caracas Arepa Bar
2                                   Teddy's Bar & Grill
3                                       Full Circle Bar
4                                       D.O.C. Wine Bar
5                                           Dim Sum Bar
6                                           North 4 Bar
7                                             Dardy Bar
8                                     Momofuku Milk Bar
9                                      Woodhul Wine Bar
10                                          The Bar 245
11                                      Fresh Kills Bar
12                                            Baker Bar
13                            Fabbrica Restaurant & Bar
14                                   Pinkerton Wine Bar
15                                     Sort Of Wine Bar
16                                              Tea Bar
17                           Brew York City Grow

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Williamsburg Brooklyn

# add a red circle marker to represent Williamsburg Brooklyn
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Brooklyn',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Bars as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## 2. Explore a Given Venue
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

In [11]:
venue_id = '49bb36fcf964a520dc531fe3' # ID of Caracas Arepa Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/49bb36fcf964a520dc531fe3?client_id=CVFZIHHKFIYX5VCCCJJLKGBEKGTCXA2VLQOCMFTXJNFRMIS2&client_secret=IHUGWTRHYEUYIZOP1UAMJKI1FAWCPHI5H22SAHAR1WGF1TXZ&v=20180604'

#### Send GET request for result

In [12]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '49bb36fcf964a520dc531fe3',
 'name': 'Caracas Arepa Bar',
 'contact': {'phone': '7182186050',
  'formattedPhone': '(718) 218-6050',
  'twitter': 'caracasarepabar',
  'instagram': 'caracasarepabar',
  'facebook': '117767434910182',
  'facebookUsername': 'CaracasArepaBar',
  'facebookName': 'Caracas Arepa Bar'},
 'location': {'address': '291 Grand St',
  'crossStreet': 'btwn Roebling & Havemeyer St',
  'lat': 40.71307533802476,
  'lng': -73.9574621701832,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71307533802476,
    'lng': -73.9574621701832}],
  'postalCode': '11211',
  'cc': 'US',
  'city': 'Brooklyn',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['291 Grand St (btwn Roebling & Havemeyer St)',
   'Brooklyn, NY 11211',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/caracas-arepa-bar/49bb36fcf964a520dc531fe3',
 'categories': [{'id': '4bf58dd8d48988d152941735',
   'name': 'Arepa Restaurant',
   'pluralName': 'Arepa Restaurants'

### B. Get the venue's overall rating

In [13]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.1


That is a very good rating. Let's check the rating of the second closest Bar.

In [14]:
venue_id = '4581734ff964a520653f1fe3' # ID of Surf Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.7


This is not a very good rating.  Let's check the rating of the third closest Bar

In [15]:
venue_id = '4262f880f964a52025211fe3' # ID of Teddy's Bar and Grill
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.5


Since this Bar's rating is lower than our first, lets focus on the first Bar.

In [16]:
venue_id = '49bb36fcf964a520dc531fe3' # ID of Caracas Arepa Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)


result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.1


### C. Get the number of tips

In [17]:
result['response']['venue']['tips']['count']

213

### D. Get the venue's tips
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

In [18]:
## Caracas Arepa Bar Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5da078c85d891b002c39a3aa'},
 'response': {'tips': {'count': 213,
   'items': [{'id': '5907461ef0ca95138496f5de',
     'createdAt': 1493648926,
     'text': 'Delicious arepas, addictive tequenos along with a "we\'re not playing" rum list and a tasty michelada, Caracas is a lasting favorite. The back yard is a fine place in summer.',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5907461ef0ca95138496f5de',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 7,
     'disagreeCount': 0,
     'lastVoteText': 'Upvoted 1 week ago',
     'lastUpvoteTimestamp': 1569628338,
     'todo': {'count': 0},
     'user': {'id': '142228',
      'firstName': 'Marius',
      'lastName': 'Watz',
      'gender': 'male',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/142228-OTAVIHUMKIDNVHV2.jpg'}},
     'authorInteractionType': 'liked'},
    {'id': '576c41bf498e648e

#### Get tips and list of associated features

In [19]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'lastVoteText', 'lastUpvoteTimestamp', 'todo', 'user', 'authorInteractionType'])

#### Format column width and display all tips

In [20]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Delicious arepas, addictive tequenos along with a ""we're not playing"" rum list and a tasty michelada, Caracas is a lasting favorite. The back yard is a fine place in summer.",7,0,5907461ef0ca95138496f5de,Marius,Watz,male,142228
1,"Consistently delicious - been here 20+ times. Pro tips: Get 3 arepas per 2 people, get the guac and the Yoyos to start with. Get a pitcher of the passionfruit drink to wash it down!!",6,0,576c41bf498e648e898aaa59,Leemor,Yuravlivker,female,20018548


Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips.

## 3. Search a Foursquare User
> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### Define URL, send GET request and display features associated with user

In [21]:
user_id = '142228' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'superuser', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

In [22]:
print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])

First Name: Marius
Last Name: Watz
Home City: Oslo


#### How many tips has this user submitted?

In [23]:
user_data['tips']

{'count': 147}

Wow! So it turns out that Marius is a very active Foursquare user, with more than 147 tips.

### Get User's tips

In [24]:
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,Kongen is the first of the Oslo ferries to switch from gas to exclusively using electric power. So now it’s even easier to take a nap.,0,0,5d9450569dccf90006f6238e
1,"It’s small and not particularly cheap. But hey, it’s local.",0,0,5d80d54092667b0006d5619e
2,"Delicious coffee, amazing Frühstück. Get the breakfast for 2 and choose 5 types of spread FTW!",0,0,5c24c33cc97f2800395b053a
3,"Delightful tapas, the black pudding croquetas were amazing.",1,0,5be0c5f9dd12f8002c227040
4,"Located in Vulkan, an industrial area redesigned as a hub for food and nightlife, Mathallen offers a mix of restaurants. Also a great spot to see the Akerselva river on your way to Grünerløkka.",2,0,5bdc35c06336be002c9873f4
5,"Charmingly corny museum dedicated to the voyages of Norwegian hero Thor Heyerdahl. Get a combo ticket and check out the Fram ship next door, too.",0,0,5bdc350cb9a5a8002c112f4a
6,"An excellent piece of urban design in the middle of the so-so Barcode real estate development, Sørenga is a great place to hang out by the water and even swim if weather allows.",0,0,5bdc348a6336be002c983cb0
7,"Delicious cocktail bar with impeccably courteous bartenders. The venue itself is draped in plants, including the herbs they use in the drinks.",0,0,5bdb474e32b61d002ca34a3e
8,"Building is from the 1600's, with a great big fireplace and cozy interior. Nice and quiet, it's perfect for a civilized cup of coffee but no guarantees on the food.",0,0,5bdb460fb9a5a8002cef8bcd
9,Cute cafe slash drinking hole on the small idyllic island of Gressholmen. Grab a beer here and combine it with some swimming if the weather permits. (Seasonal.),0,0,5bdb4459340a58002c89cb82


#### Let's get the venue for the tip with the greatest number of agree counts

In [25]:
tip_id = '5bdb2bbc1af852002cd67bb0' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

Nesoddbåten
{'lat': 59.910563054562544, 'lng': 10.729425720634632, 'labeledLatLngs': [{'label': 'display', 'lat': 59.910563054562544, 'lng': 10.729425720634632}], 'postalCode': '0256', 'cc': 'NO', 'city': 'Oslo', 'state': 'Oslo', 'country': 'Norge', 'formattedAddress': ['0256 Oslo', 'Norge']}


### Get User's friends

In [26]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

,bio,firstName,gender,homeCity,id,lastName,lists.groups,photo.prefix,photo.suffix,superuser,tips.count
0,"artist, game designer, social network enthusiasts, activist, Homo sapien",Paba,female,"london, uk",161932,B,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/KQODFSEX02EBPRAK.jpg,1.0,70
1,tall with short legs.,Scott,male,,2155,Fitzgerald,"[{'type': 'created', 'count': 24, 'items': []}]",https://fastly.4sqi.net/img/user/,/-SBP1YXSXHJFIOHVO.jpg,5.0,59
2,,Simon,male,"London, UK",11070394,Sanky,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/ZQRTNRXWVMXVMYSB.jpg,NaN,0
3,What now for man raised by puffins?,Julian,male,London,513787,Burgess,"[{'type': 'created', 'count': 3, 'items': []}]",https://fastly.4sqi.net/img/user/,/0QNWZERWCIU4HKM5.jpg,NaN,27
4,,Mrcl,none,Berlin,29116944,Schwttlck,"[{'type': 'created', 'count': 4, 'items': []}]",https://fastly.4sqi.net/img/user/,/BCZQONIARMRWHT0T.jpg,NaN,3
5,,Joseph,male,"Chicago, IL",37839440,Chiocchi,"[{'type': 'created', 'count': 3, 'items': []}]",https://fastly.4sqi.net/img/user/,/FDDUB1ECV3RNOPLG.jpg,NaN,0
6,,P.,none,Lille,22096120,Box,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/MSYBZV3WMTPUWWO0.png,NaN,0
7,,Chris,female,"San Antonio, Texas",23287719,Davila,"[{'type': 'created', 'count': 3, 'items': []}]",https://fastly.4sqi.net/img/user/,/23287719-XCBEFZGN5XFAPUO1.jpg,NaN,5
8,,Valentín,male,"Granada, España",6355781,Pedrosa,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/OET130BD5SXOW3ZU.jpg,NaN,2
9,Yes.,Joshua,male,"Manhattan, NY",39727,Goldberg,"[{'type': 'created', 'count': 2, 'items': []}]",https://fastly.4sqi.net/img/user/,/39727_1253904304485.png,2.0,8


it appears that Marius has many friends

### Retrieve the User's Profile Image

In [27]:
user_data

{'id': '142228',
 'firstName': 'Marius',
 'lastName': 'Watz',
 'gender': 'male',
 'canonicalUrl': 'https://foursquare.com/user/142228',
 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
  'suffix': '/142228-OTAVIHUMKIDNVHV2.jpg'},
 'friends': {'count': 163,
  'groups': [{'type': 'others',
    'name': 'Other friends',
    'count': 163,
    'items': [{'id': '161932',
      'firstName': 'Paba',
      'lastName': 'B',
      'gender': 'female',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/KQODFSEX02EBPRAK.jpg'},
      'tips': {'count': 70},
      'lists': {'groups': [{'type': 'created', 'count': 2, 'items': []}]},
      'homeCity': 'london, uk',
      'bio': 'artist, game designer, social network enthusiasts, activist, Homo sapien',
      'contact': {},
      'superuser': 1},
     {'id': '2155',
      'firstName': 'Scott',
      'lastName': 'Fitzgerald',
      'gender': 'male',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
      

In [28]:
# 1. grab prefix of photo
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://fastly.4sqi.net/img/user/3000x3000/142228-OTAVIHUMKIDNVHV2.jpg')

## 4. Explore a location
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

#### So, you just finished your drink at Caracas Arepa Bar, and are just curious about the popular spots around the bar. In order to explore the area, let's start by getting the latitude and longitude values of Caracas Arepa Barz.

In [29]:
latitude = 40.71307533802476
longitude = -73.9574621701832

In [30]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=CVFZIHHKFIYX5VCCCJJLKGBEKGTCXA2VLQOCMFTXJNFRMIS2&client_secret=IHUGWTRHYEUYIZOP1UAMJKI1FAWCPHI5H22SAHAR1WGF1TXZ&ll=40.71307533802476,-73.9574621701832&v=20180604&radius=500&limit=30'

#### Send GET request and examine results

In [31]:
import requests

In [32]:
results = requests.get(url).json()
'There are {} around Caracas Arepa Bar.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Caracas Arepa Bar.'

#### Get relevant part of JSON

In [33]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4e4ea8b5aeb70f12849528ed',
  'name': 'St. Mazie',
  'location': {'address': '345 Grand St',
   'crossStreet': 'btwn Marcy Ave & Havemeyer St',
   'lat': 40.712446182313414,
   'lng': -73.95595408442468,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.712446182313414,
     'lng': -73.95595408442468}],
   'distance': 145,
   'postalCode': '11211',
   'cc': 'US',
   'city': 'Brooklyn',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['345 Grand St (btwn Marcy Ave & Havemeyer St)',
    'Brooklyn, NY 11211',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d11e941735',
    'name': 'Cocktail Bar',
    'pluralName': 'Cocktail Bars',
    'shortName': 'Cocktail',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/cocktails_',
     'suffix': '.png'},
    'primary': Tru

#### Process JSON and convert it to a clean dataframe

In [34]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,St. Mazie,Cocktail Bar,345 Grand St,US,Brooklyn,United States,btwn Marcy Ave & Havemeyer St,145,"[345 Grand St (btwn Marcy Ave & Havemeyer St), Brooklyn, NY 11211, United States]","[{'label': 'display', 'lat': 40.712446182313414, 'lng': -73.95595408442468}]",40.712446,-73.955954,NaN,11211,NY,4e4ea8b5aeb70f12849528ed
1,East River Pilates,Pilates Studio,235 S 1st St,US,Brooklyn,United States,at Roebling street,89,"[235 S 1st St (at Roebling street), Brooklyn, NY 11211, United States]","[{'label': 'display', 'lat': 40.71289117042526, 'lng': -73.95850006447007}]",40.712891,-73.958500,NaN,11211,NY,54df7b38498eee7bd051f0bc
2,L'Industrie Pizzeria,Pizza Place,254 S 2nd St,US,Brooklyn,United States,Havemeyer,157,"[254 S 2nd St (Havemeyer), Brooklyn, NY 11211, United States]","[{'label': 'display', 'lat': 40.7116752206617, 'lng': -73.95769662100513}]",40.711675,-73.957697,NaN,11211,NY,55135426498e9dee8f19cc5f
3,St. Anselm,American Restaurant,355 Metropolitan Ave,US,Brooklyn,United States,btwn Havemeyer & Roebling St,172,"[355 Metropolitan Ave (btwn Havemeyer & Roebling St), Brooklyn, NY 11211, United States]","[{'label': 'display', 'lat': 40.71414471294114, 'lng': -73.95598405757656}]",40.714145,-73.955984,NaN,11211,NY,4be5bbe8d4f7c9b6b9d52520
4,Emmy Squared,Pizza Place,364 Grand St,US,Brooklyn,United States,at Marcy Ave,179,"[364 Grand St (at Marcy Ave), Brooklyn, NY 11211, United States]","[{'label': 'display', 'lat': 40.712166, 'lng': -73.955705}]",40.712166,-73.955705,NaN,11211,NY,570139a9498e16178005c729
5,Twenty Sided Store,Toy / Game Store,362 Grand St,US,Brooklyn,United States,at Marcy St,190,"[362 Grand St (at Marcy St), Brooklyn, NY 11211, United States]","[{'label': 'display', 'lat': 40.712268329253185, 'lng': -73.9554761947898}]",40.712268,-73.955476,NaN,11211,NY,4d9764eb942ba093f86e6b8c
6,Have & Meyer Chatteria,Wine Bar,103 Havermeyer St,US,Brooklyn,United States,Hope St,88,"[103 Havermeyer St (Hope St), Brooklyn, NY 11211, United States]","[{'label': 'display', 'lat': 40.71330486799995, 'lng': -73.95646386109051}]",40.713305,-73.956464,NaN,11211,NY,563169d2498e609fa36f7bd3
7,The Four Horsemen,Wine Bar,295 Grand St,US,Brooklyn,United States,btwn Havemeyer & Roebling St,15,"[295 Grand St (btwn Havemeyer & Roebling St), Brooklyn, NY 11211, United States]","[{'label': 'display', 'lat': 40.71297062908573, 'lng': -73.9573489459191}]",40.712971,-73.957349,NaN,11211,NY,554f66e8498e387866311421
8,Bozu,Sushi Restaurant,296 Grand St,US,Brooklyn,United States,btwn Roebling & Havemeyer St,11,"[296 Grand St (btwn Roebling & Havemeyer St), Brooklyn, NY 11211, United States]","[{'label': 'display', 'lat': 40.71297626571757, 'lng': -73.95747790354523}]",40.712976,-73.957478,NaN,11211,NY,423e0e80f964a52048201fe3
9,Caracas Arepa Bar,Arepa Restaurant,291 Grand St,US,Brooklyn,United States,btwn Roebling & Havemeyer St,0,"[291 Grand St (btwn Roebling & Havemeyer St), Brooklyn, NY 11211, United States]","[{'label': 'display', 'lat': 40.71307533802476, 'lng': -73.9574621701832}]",40.713075,-73.957462,NaN,11211,NY,49bb36fcf964a520dc531fe3


#### Let's visualize these items on the map around our location

In [35]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

## 5. Explore Trending Venues
> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

#### Now, instead of simply exploring the area around Caracas Arepa Bar, you are interested in knowing the venues that are trending at the time when people are out at the bars, meaning the places with the highest foot traffic. So let's do that and get the trending venues around Caracas Arepa Bar.

In [36]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5da078ca598e64002c23853a'},
 'response': {'venues': []}}

### Check if any venues are trending at this time

In [37]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [38]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 

### Visualize trending venues

In [39]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Caracas Arepa Bar as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Caracas Arepa Bar',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [40]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Brooklyn</font></h1>

## Introduction

In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in New York City. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [42]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

In [43]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [46]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [47]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [48]:
neighborhoods_data = newyork_data['features']

In [49]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [50]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [51]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [53]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [54]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [56]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 612 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In [57]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [58]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [59]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [61]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [62]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.